# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# your code here
import pandas as pd

df = pd.read_csv('tic-tac-toe.csv')

print('Data Head:\n', df.head())
print('\nData Shape:\n', df.shape)
print('\nData Describe:\n', df.describe())
print('\nMissing Values:\n', df.isnull().sum())
print('\nData types:\n', df.dtypes)


Data Head:
   TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True

Data Shape:
 (958, 10)

Data Describe:
          TL   TM   TR   ML   MM   MR   BL   BM   BR class
count   958  958  958  958  958  958  958  958  958   958
unique    3    3    3    3    3    3    3    3    3     2
top       x    x    x    x    x    x    x    x    x  True
freq    418  378  418  378  458  378  418  378  418   626

Missing Values:
 TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

Data types:
 TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object


In [2]:

# Convert the categorical values to numeric in all columns.

df_encoded = pd.get_dummies(df, columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class'])

df_encoded.head()


,TL_b,TL_o,TL_x,TM_b,TM_o,TM_x,TR_b,TR_o,TR_x,ML_b,...,BL_o,BL_x,BM_b,BM_o,BM_x,BR_b,BR_o,BR_x,class_False,class_True
0,False,False,True,False,False,True,False,False,True,False,...,False,True,False,True,False,False,True,False,False,True
1,False,False,True,False,False,True,False,False,True,False,...,True,False,False,False,True,False,True,False,False,True
2,False,False,True,False,False,True,False,False,True,False,...,True,False,False,True,False,False,False,True,False,True
3,False,False,True,False,False,True,False,False,True,False,...,True,False,True,False,False,True,False,False,False,True
4,False,False,True,False,False,True,False,False,True,False,...,False,False,False,True,False,True,False,False,False,True


In [4]:

#separate the inputs and output

X = df_encoded.drop(columns=['class_True', 'class_False'])  # Inputs: all except the target columns
y = df_encoded['class_True']  # Output: binary target (class_True)

# Print the first few rows to verify
print('\nX Head:\n', X.head())
print('\nTarget y Head:\n', y.head())


#normalize the data the input data

from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler to X and transform X
X_normalized = scaler.fit_transform(X)

# Optionally, convert the normalized data back to a DataFrame
X_normalized = pd.DataFrame(X_normalized, columns=X.columns)

# Verify the normalization
print(X_normalized.head())




X Head:
     TL_b   TL_o  TL_x   TM_b   TM_o  TM_x   TR_b   TR_o  TR_x   ML_b  ...  \
0  False  False  True  False  False  True  False  False  True  False  ...   
1  False  False  True  False  False  True  False  False  True  False  ...   
2  False  False  True  False  False  True  False  False  True  False  ...   
3  False  False  True  False  False  True  False  False  True  False  ...   
4  False  False  True  False  False  True  False  False  True  False  ...   

    MR_x   BL_b   BL_o   BL_x   BM_b   BM_o   BM_x   BR_b   BR_o   BR_x  
0  False  False  False   True  False   True  False  False   True  False  
1  False  False   True  False  False  False   True  False   True  False  
2  False  False   True  False  False   True  False  False  False   True  
3  False  False   True  False   True  False  False   True  False  False  
4  False   True  False  False  False   True  False   True  False  False  

[5 rows x 27 columns]

Target y Head:
 0    True
1    True
2    True
3    True
4  

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [5]:
# your code here
#split the data into training and testing data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)







X_train shape: (670, 27)
X_test shape: (288, 27)
y_train shape: (670,)
y_test shape: (288,)


In [6]:
# create a sequential model
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

# add several layers to model Relu activation function for middle layers and sothmax for output layer

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(20, activation='softmax')) 








2024-10-07 19:01:05.207977: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# compile the model 

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
#fit the training data to the model
model.fit(X_train, y_train, epochs=100, batch_size=10)

#evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

#save the model tic-tac-toe.model
model.save('tic-tac-toe.keras')




Epoch 1/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 4.7544e-05
Epoch 2/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 4.1729e-05
Epoch 3/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 5.3416e-05
Epoch 4/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 4.3604e-05
Epoch 5/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 3.9041e-05
Epoch 6/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 4.6462e-05
Epoch 7/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 3.4782e-05
Epoch 8/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 3.7347e-05
Epoch 9/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 2.9915e-05
Epoch 10/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 4.3908e-05
Epoch 11/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 2.7921e-05
Epoch 12/100
67/67 

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [15]:
# your code here

# load the model and make a prediction on a few random rows of the testing data set

from keras.models import load_model

model = load_model('tic-tac-toe.keras')

predictions = model.predict(X_test[:5])
print('Predictions:', predictions)
print('Actual:', y_test[:5])

print('Loss:', loss)
print('Accuracy:', accuracy)




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Predictions: [[9.99998808e-01 1.21164157e-06 1.17257537e-09 1.04843290e-09
  1.06021303e-09 2.27796310e-10 3.04640491e-09 7.46223305e-10
  5.12185072e-10 3.89698857e-10 6.72540912e-10 5.16859333e-10
  4.18046220e-09 4.27051311e-10 5.87501992e-10 3.71828429e-10
  1.98496286e-09 2.30055175e-09 1.87335214e-09 8.32556413e-10]
 [4.25119575e-12 1.00000000e+00 3.05437468e-12 3.59492622e-12
  5.41220931e-12 2.32209499e-12 1.15343256e-11 5.00137215e-12
  4.08206281e-12 6.60550130e-12 6.20435951e-12 1.29164023e-11
  3.55819822e-12 1.72028440e-12 1.33034295e-11 9.62825045e-12
  1.91063988e-11 1.03245815e-11 1.59104310e-11 6.73350958e-12]
 [4.45420301e-05 9.99955058e-01 1.64953704e-08 1.07772324e-08
  2.21519869e-08 4.87088148e-09 4.08996961e-08 1.50749120e-08
  1.31913245e-08 1.70037069e-08 2.39241107e-08 4.81501807e-08
  2.57460098e-08 4.30121982e-09 2.79374355e-08 2.61465605e-08
  2.10475353e-08 2.75491434e-08 2.40176998e-08 1.43645931e-08]
 [1.00000000e+00

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [14]:
# add a few more layers to the model and see if the accuracy improves
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(20, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=10)

loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)






Epoch 1/100


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5420 - loss: 1.9100
Epoch 2/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6891 - loss: 0.5739
Epoch 3/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7873 - loss: 0.4854
Epoch 4/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7948 - loss: 0.4553
Epoch 5/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8393 - loss: 0.3507
Epoch 6/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9177 - loss: 0.2335
Epoch 7/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9748 - loss: 0.1138
Epoch 8/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9822 - loss: 0.0723
Epoch 9/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9891 - loss: 0.0440
Epoch 10/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9912 - loss: 0.0310
Epoch 11/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9999 - loss: 0.0111
Epoch 12/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9965 - lo

**Which approach(es) did you find helpful to improve your model performance?**

# your answer here

The first as a lower Loss perfomed better then the second model with more layers but i cant understand, why 



First Model 

Loss: 0.038903698325157166 
Accuracy: 0.9895833134651184

Second model:

Loss: 0.0646730288863182
Accuracy: 0.9861111044883728
